In [1]:
from gazpacho import get, Soup
import csv
import os
import pandas as pd
import numpy as np
from itertools import groupby
import pathlib
import os
import math

In [2]:
url = "http://rosalind.info/glossary/rna-codon-table/"
html = get(url)
soup = Soup(html)
glossary_terms = soup.find("div", {"class": "glossary-term"})
RNA_codon_table = glossary_terms.find("pre").text

RNA_codon_table = RNA_codon_table.replace("\n", ",").replace("     ", ",").replace("    ", ",").replace("      ", ",")
RNA_codon_table = RNA_codon_table.replace("   ", ",")
RNA_codon_table = RNA_codon_table.replace(", ", ",")
RNA_codon_table = RNA_codon_table.replace(" ", ":").split(",")

gene_to_amino_code = {}

for code in RNA_codon_table:
    code = code.split(":")
    if code[0] in gene_to_amino_code.keys():
        gene_to_amino_code[code[0].replace("U", "T")].append(code[1])
    else:
        gene_to_amino_code[code[0].replace("U", "T")] = []
        gene_to_amino_code[code[0].replace("U", "T")].append(code[1])

In [3]:
def get_rosalind_data(filepath, filename, readlines=False):    
    
    if readlines:
        with open(filepath+filename, "r") as f:
            data = f.readlines()
            data = [line.strip("\n") for line in data]
    else: 
        with open(filepath+filename, "r") as f:
            data = f.read().strip("\n")

    return data

In [4]:
filepath="C:\\Users\\adeoy\\Downloads\\"
filename="rosalind_orf (2).txt"

data = get_rosalind_data (filepath, filename, readlines=True)

RNA = ""
for item in data[1:]:
    RNA += item


In [5]:
RNA_comp = "".join(["T" if base=="A" else "C" if base=="G" else "G" if base=="C"
                            else "A" if base=="T" else "" for base in RNA])

In [6]:
RNA_prob = [RNA[:len(RNA)-len(RNA)%3],
            RNA_comp[:len(RNA)-len(RNA)%3][::-1],
            RNA[1:][:len(RNA[1:])-len(RNA[1:])%3], 
            RNA_comp[1:][:len(RNA[1:])-len(RNA[1:])%3][::-1],
            RNA[2:][:len(RNA[2:])-len(RNA[2:])%3], 
            RNA_comp[2:][:len(RNA[2:])-len(RNA[2:])%3][::-1],]

In [7]:
k = 0
forward_reading_frame = [] 
for RNA in RNA_prob:
    counter = 0
    protein = ""
    start_codon_locations = []
    stop_codon_locations = []
    k+=1
    counter = 0
    for i in range(len(RNA)//3): 
        
        if gene_to_amino_code[RNA[counter:counter+3]][0] == "M":
            start_codon_locations.append(i)
        elif gene_to_amino_code[RNA[counter:counter+3]][0] == "Stop":
            stop_codon_locations.append(i)
            
        protein += gene_to_amino_code[RNA[counter:counter+3]][0]
        protein = protein.replace("Stop", "s")
        counter += 3
    
    
    for start_pos in start_codon_locations: 
        for stop_pos in stop_codon_locations:
            if start_pos < stop_pos:
                break
            
        if ("s" in protein) and (protein[start_pos:stop_pos] != "") and protein[start_pos:stop_pos] not in forward_reading_frame:
                
            forward_reading_frame.append(protein[start_pos:stop_pos]) 


In [8]:
for frame in forward_reading_frame:
    print(frame)

M
MSDGK
MHVIQSLSRYCKGPEVLDTQK
MARILINDLGSWLANHEPRSYS
MSLPRLHTGLAFARNEIHQSKWRIKRPRGCSRIILPYCKPACTSGIYFIVASGSLQLDLQGTRPRYRVCF
MDFVPGES
MLDLDRGPLIGRPPLLHRATGR
MISVRG
MYSNSTPKEILVLRSTGNLKRVDN
MSMISVRG
MYGRRSNL
MGSGNFSTTSRPT
MEAAL
MLQSVSEFYRLDRLPYIRYLSFSGLTTTNTHARHSILVQVLQGPRGTRYTKIRSAPRKKIELLNMPLAVPHGEDLN
MPLAVPHGEDLN
MK
MSLVYEYDLGSWLANHEPRSLIKILAMRDCEWHI
MRDCEWHI
MTCMCIGRG
MCIGRG
